![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

# Topic-Model Tool

This notebook-based tool accompanies the [Language Technology and Data Analysis Laboratory (LADAL) tutorial *Topic Modelling R*](https://ladal.edu.au/topicmodels.html). 

## Using your own data

<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
To <b>use your own data</b>, click on the folder called <b>`MyTexts`</b> (it is in the menu to the left of the screen) and then simply drag and drop your txt-files into the folder. <br>When you then execute the code chunk below, you will upload your own data and you can then use it in this notebook.<br>You can upload <b>only txt-files</b> (simple unformatted files created in or saved by a text editor)! <br>The notebook assumes that you upload some form of text data - not tabular data!<br><br>
<b>IMPORTANT: topic modeling works best if the texts are neither too short nor too long. Typically  paragraphs work best (texts above sentence level and below article length).</b>
<br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


In [ ]:
# load function that helps loading texts
source("https://slcladal.github.io/rscripts/loadtxts.R")
# load texts
corpus <- loadtxts("notebooks/MyTexts")
# inspect the structure of the text object
str(corpus)


## Cleaning and tokenising

We start by cleaning the corpus data (by removing tags, artefacts and non-alpha-numeric characters) and then splitting the clean corpora into individual words.


In [ ]:
# load function that helps loading texts
source("https://slcladal.github.io/rscripts/preptop.R")
# clean corpus
clean_dfm <- preptop(corpus)


## Unsupervised LDA

Now that we have cleaned the data, we can perform the topic modelling. This consists of two steps:

1. First, we perform an unsupervised LDA. We do this to check what topics are in our corpus. 

2. Then, we perform a supervised LDA (based on the results of the unsupervised LDA) to identify meaningful topics in our data. For the supervised LDA, we define so-called *seed terms* that help in generating coherent topics.


<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>In the code below, the default is to look for 10 topics but you will need to vary and adapt the number of topics (k) for your data and check what topics are in your data.</b>
<br>
</p></span>
</div>

<br>


In [ ]:
# generate model: change k to different numbers, e.g. 5 or 15 and look for consistencies in the keywords for the topics below.
tmlda <- topicmodels::LDA(clean_dfm, k = 10, control = list(seed = 1234))


Now that the topic model has been generated, you can inspect it. The code chunk below will show the top 10 terms for each topic.



In [ ]:
# load function that tabulates top terms of topics
source("https://slcladal.github.io/rscripts/tabtop.R")
# inspect
tabtop(tmlda, 10)


## Supervised LDA

Now, we perform a supervised LDA. Here we use the keywords extracted based on the unsupervised LDA as *seed terms* for topics to create coherent topics.

<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>IMPORTANT: you need to change and adapt the topics and keywords defined below </b> <br>Simply replace the topics and seed terms with your own topics and seed terms (based on the results of the unsupervised LDA!). 
<br>
</p>
</span>
</div>

<br>


In [ ]:
# semisupervised LDA
dict <- dictionary(list(Computer = c("computers", "information", "machine", "computer"),
                        Education = c("students", "courses", "education", "university"),
                        Movies = c("movie", "film", "commercial", "watch")))
tmod_slda <- seededlda::textmodel_seededlda(clean_dfm, 
                                            dict, 
                                            residual = TRUE, 
                                            min_termfreq = 5)
terms(tmod_slda)


Now, we extract files and create a data frame of topics and documents. This shows what topic is dominant in which file in tabular form.  



In [ ]:
files <- stringr::str_replace_all(names(topics(tmod_slda)), ".*/(.*?).txt", "\\1")
cleancontent <- corpus
topics <- topics(tmod_slda)
# generate data frame
df <- data.frame(files, cleancontent, topics) %>%
  dplyr::mutate_if(is.character, factor)
# inspect
head(df)


## Exporting data

To export a data frame as an MS Excel spreadsheet, we use `write_xlsx`. Be aware that we use the `here` function to  save the file in the current working directory.


In [ ]:
# save data for MyOutput folder
write_xlsx(dfp, here::here("notebooks/MyOutput/df.xlsx"))


<div class="warning" style='padding:0.1em; background-color: rgba(215,209,204,.3); color:#51247a'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>You will find the generated MS Excel spreadsheet named "df.xlsx" in the `MyOutput` folder (located on the left side of the screen).</b> <br><br>Simply double-click the `MyOutput` folder icon, then right-click on the "df.xlsx" file, and choose Download from the dropdown menu to download the file. <br>
</p>
<p style='margin-left:1em;'>
</p></span>
</div>

<br>


***

# Citation & Session Info

Schweinberger, Martin. (2024)` *LADAL Topic-Model Tool*. Brisbane: The University of Queensland. url: https://ladal.edu.au/tools.html (Version 2024.04.21).


In [ ]:
@manual{schweinberger2024topictool,
  author = {Schweinberger, Martin},
  title = {LADAL Topic-Model Tool},
  note = {https://ladal.edu.au/tools.html},
  year = {2024},
  organization = {The University of Queensland, School of Languages and Cultures},
  address = {Brisbane},
  edition = {2024.04.21}
}


[Back to LADAL](https://ladal.edu.au/topicmodels.html)

***


In [ ]:
sessionInfo()

